# Importing

In [3]:
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from sklearn.preprocessing import StandardScaler # om een of andere reden, wat ik ook probeer (believe me ik ben al een uur bezig) wilt deze t niet doen
from sklearn.feature_selection import VarianceThreshold

ModuleNotFoundError: No module named 'sklearn'

# Reading data


In [ ]:
data_raw = pd.read_csv('tested_molecules.csv')

# Calculating descriptors

In [ ]:
def calc_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles) #string to molecule object
    descriptors = {}
    for name, func in Descriptors.descList:
        descriptors[name] = func(mol)
    return descriptors


descriptor_data =data_raw['SMILES'].apply(calc_descriptors) #blabla kijk ff chatgpt, willen we losse df of dit als rij? vgm asl rij, check ff Evi's versie 

# Adding data to a dataframe

In [ ]:
df_molecules = pd.DataFrame(data_raw['SMILES']) #create a new dataframe for the smiles and descriptors

df_descriptors=pd.DataFrame(descriptor_data.tolist()) #making a dataframe for the descriptor data

df_molecules = pd.concat([df_molecules, df_descriptors], axis=1)

df_molecules.head()


,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,C=C(C)c1nc(N)nc(N)n1,5.313889,0.120833,5.313889,0.120833,0.592228,151.173,142.101,151.085795,58,...,0,0,0,0,0,0,0,0,0,0
1,C=C(Cl)COc1ccc2c(C)cc(=O)oc2c1,11.238954,-0.366756,11.238954,0.225308,0.785414,250.681,239.593,250.039672,88,...,0,0,0,0,0,0,0,0,0,0
2,C=CCNC(=O)CCCC(=O)NCC=C,11.090706,-0.049610,11.090706,0.049610,0.581062,210.277,192.133,210.136828,84,...,0,0,0,0,0,0,0,0,0,0
3,C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21,11.892238,-0.457824,11.892238,0.076632,0.441090,232.239,220.143,232.084792,88,...,0,0,0,0,0,0,0,0,0,0
4,C=CCn1cc(Cl)c(=O)n(CC=C)c1=O,11.693580,-0.498260,11.693580,0.012315,0.720343,226.663,215.575,226.050905,80,...,0,0,0,0,0,0,0,0,0,0


# Splitting and scaling the descriptors

In [ ]:
# For the filters to work it is better if all data is standardized so we will make a copy so we don't lose the actual values

df_molecules_copy = df_molecules.copy()
df_descriptors=df_molecules_copy.drop(columns=['SMILES'])
# We do not want to standardize binary columns, so we need to seperate these from the other columns
binary_columns = []#list of names of the binary columns
non_binary_columns=[]

for column in df_descriptors.columns:
    if df_molecules_copy[column].nunique() == 2 and df_descriptors[column].min() == 0 and df_descriptors[column].max() == 1:
        binary_columns.append(column)
    elif column not in binary_columns:
        non_binary_columns.append(column)

print(binary_columns)
print(non_binary_columns)

#oke sklearn wilde t opeens niet doen in deze environment, maar rdkit niet in de ander, vanaf hier kan ik mijn code dus neit meer testen, ik hoop dat ie t doet bij jullie, anders lmk :))
binary_descriptors=df_descriptors[binary_columns]
non_binary_descriptors=df_descriptors[non_binary_columns]

scaler = StandardScaler()
standardized_non_binary_cols=pd.DataFrame(scaler.fit_transform(non_binary_descriptors),
                                                   columns=non_binary_columns.columns)

#put the binary columns together with the standardized non binary columns
df_standardized_descriptors = pd.concat([standardized_non_binary_cols, binary_descriptors], axis=1) #put the binary columns together with the standardized non binary columns




['fr_Al_COO', 'fr_C_S', 'fr_Imine', 'fr_barbitur', 'fr_dihydropyridine', 'fr_guanido', 'fr_lactam', 'fr_lactone', 'fr_nitro', 'fr_nitro_arom', 'fr_oxazole', 'fr_priamide']
['SMILES', 'MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 

# Applying filters

In [ ]:
#Low variance filter:
var_filter = VarianceThreshold(threshold=0.001)
X_var_filtered = var_filter.fit_transform(df_standardized_descriptors)
df_filtered_once_descriptors = pd.DataFrame(X_var_filtered, columns=df_standardized_descriptors.columns[var_filter.get_support()])


#high-correlation filter
# Calculate the correlation matrix
corr_matrix = df_filtered_once_descriptors.corr().abs()

# Flatten the correlation matrix to get unique pairs of columns
corr_values = corr_matrix.unstack()

# Drop the diagonal values (correlation of a feature with itself)
corr_values = corr_values.drop(labels=corr_matrix.columns)

# Get pairs of columns with correlation greater than the threshold (e.g., 0.9)
highly_correlated_pairs = corr_values[corr_values > 0.95].index.tolist()

# Extract unique columns from the pairs
to_drop = set()
for pair in highly_correlated_pairs:
    to_drop.add(pair[0])  # Add the first column of the pair
    to_drop.add(pair[1])  # Add the second column of the pair (both need to be dropped since correlation is the same both ways)

# Drop highly correlated columns from the filtered descriptors DataFrame
df_filtered_twice_descriptors = df_filtered_once_descriptors.drop(columns=to_drop)


# We zouden evt nog PCA kunnen doen? Ik weet ff neit of dat hierbij nu nut heeft of niet




# Getting filtered columns from original data

In [ ]:
#OKay nu hebben we een df die gefilterd is, maar met standardized waarden, we willen t origineel hebben dus we moeten die kolommen eruit halen:
relevant_columns=['SMILES']
filtered_descriptors=list(df_filtered_twice_descriptors.columns)

relevant_columns.append(filtered_descriptors)

df_filtered_molecules=df_molecules[relevant_columns]
df_filtered_molecules # dit is de output met smiles en dan de descriptors die relevant zijn


#test mocht sklearn het ooit doen:
#descriptors_left=list(df_filtered_molecules.columns)
#print(descriptors_left)